In [1]:
import json

import pandas as pd

import wandb


def get_tables(run_id):
    with wandb.init() as run:
        dfs = {}
        for bench in ["bioasq"]:  # ['medqa', 'mmlumed', 'medmcqa', 'bioasq']:
            artifact = run.use_artifact(
                f"liac/OntoRAG-biomed/run-{run_id}-results_{bench}:v0",
                type="run_table",
            )
            artifact_dir = artifact.download()
            dfjson = json.load(
                open(artifact_dir + f"/results_{bench}.table.json")
            )
            df = pd.DataFrame(dfjson["data"], columns=dfjson["columns"])
            dfs[bench] = df
    return dfs


# match the keys with the names
runs = {
    "63jneekd": "orag-hyq-tm",
    "3njkj2tw": "orag-tm",
    "bj77mnsq": "orag-hyq",
    "nt9v14x0": "orag-simple",
    "chgt9pvp": "cot",
    "f3rej8wv": "zeroshot",
}

for run, name in runs.items():
    dfm = get_tables(run)
    for k, df in dfm.items():
        df.to_csv(f"data/_{k}_{name}.csv", index=False, sep="\t")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: drecmb (liac). Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


wandb:   1 of 1 files downloaded.  


wandb:   1 of 1 files downloaded.  


wandb:   1 of 1 files downloaded.  


wandb:   1 of 1 files downloaded.  


wandb:   1 of 1 files downloaded.  


## Accuracy for ontorags, when context was retrieved

In [2]:
def clean_model_answer_abcde(model_out):
    """Process the model output to get the answer."""
    try:
        model_out = model_out.split("Answer:")[1].strip()
    except:
        return None
    inter = set(model_out).intersection(set("ABCDE"))
    if len(model_out) == 1:
        if model_out in list("ABCDE"):
            return model_out
    elif inter:
        if len(inter) == 1:
            return inter.pop()
        elif ":" in model_out:
            if model_out.split(":")[0] in "ABCDE":
                return model_out.split(":")[0]
    return None


def accuracy_yn(df, s):
    # preprocess choice_answer
    df["choice_answer_proc"] = (
        df["choice_answer"]
        .astype(str)
        .str.split("Answer:")
        .apply(lambda x: x[1] if len(x) > 1 else x[0])
    )
    acc = (
        df["choice_answer_proc"].str.strip().str.lower()
        == df["answer"].str.strip().str.lower()
    ).mean()
    print(f"{s} {acc*100:.4f}%")


def accuracy_abcde(df, s):
    # preprocess choice_answer
    df["choice_answer_proc"] = df["choice_answer"].apply(
        clean_model_answer_abcde
    )
    acc = (df["choice_answer_proc"] == df["answer"]).mean()
    print(f"{s} {acc*100:.4f}%")


bench = "bioasq"
accuracy_metric = accuracy_yn
zs = pd.read_csv(f"data/_{bench}_zeroshot.csv", sep="\t")
cot = pd.read_csv(f"data/_{bench}_cot.csv", sep="\t")
for name in runs.values():
    df = pd.read_csv(f"data/_{bench}_{name}.csv", sep="\t")
    print(f"\n{name}:")
    accuracy_metric(df, s="Accuracy: ")
    if "orag" in name:
        accuracy_metric(
            df[df.context.astype(str).str.len() > 2],
            s="Accuracy (if context): ",
        )
        accuracy_metric(
            zs[df.context.astype(str).str.len() > 2],
            s="Zeroshot acc on these: ",
        )
        accuracy_metric(
            cot[df.context.astype(str).str.len() > 2],
            s="CoT acc on these:      ",
        )


orag-hyq-tm:
Accuracy:  65.0446%
Accuracy (if context):  71.9588%
Zeroshot acc on these:  87.4227%
CoT acc on these:       85.7732%

orag-tm:
Accuracy:  64.2221%
Accuracy (if context):  67.0959%
Zeroshot acc on these:  86.2661%
CoT acc on these:       84.2632%

orag-hyq:
Accuracy:  83.6189%
Accuracy (if context):  83.8328%
Zeroshot acc on these:  86.2776%
CoT acc on these:       85.3312%

orag-simple:
Accuracy:  80.1919%
Accuracy (if context):  81.4020%
Zeroshot acc on these:  86.2661%
CoT acc on these:       84.2632%

cot:
Accuracy:  84.8526%

zeroshot:
Accuracy:  85.9493%


/tmp/ipykernel_2570282/1093007628.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["choice_answer_proc"] = (
/tmp/ipykernel_2570282/1093007628.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["choice_answer_proc"] = (
/tmp/ipykernel_2570282/1093007628.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

## Analyze no. concepts vs performance

In [3]:
def clean_model_answer_abcde(model_out):
    """Process the model output to get the answer."""
    try:
        model_out = model_out.split("Answer:")[1].strip()
    except:
        return None
    inter = set(model_out).intersection(set("ABCDE"))
    if len(model_out) == 1:
        if model_out in list("ABCDE"):
            return model_out
    elif inter:
        if len(inter) == 1:
            return inter.pop()
        elif ":" in model_out:
            if model_out.split(":")[0] in "ABCDE":
                return model_out.split(":")[0]
    return None

In [4]:
import numpy as np
import pandas as pd

dfm = {}
for m in ["orag-hyq-tm", "orag-tm", "orag-hyq", "orag-simple"]:
    for bench in ["medqa", "mmlumed", "medmcqa"]:
        if m not in dfm:
            dfm[m] = {}
        dfm[m][bench] = pd.read_csv(f"data/_{bench}_{m}.csv", sep="\t")

import re


def ncon(context):
    if isinstance(context, str):
        return len(re.findall(r"\"concepts\":", context))
    return 0


# Calc accuracy and # concepts for each variation, for each benchmark
res = {}
for bench in ["medqa", "mmlumed", "medmcqa"]:
    accs = []
    nconcepts = []
    for m in dfm.keys():
        ac = np.mean(
            dfm[m][bench].choice_answer.apply(clean_model_answer_abcde)
            == dfm[m][bench].answer
        )
        concepts = dfm[m][bench].context.apply(ncon)
        accs.append(ac)
        nconcepts.append(concepts.mean())

    print(f"{bench}. Corr: {np.corrcoef(accs, nconcepts)[0,1]:.4f}")
    res[bench] = (accs, nconcepts)

medqa. Corr: 0.7852
mmlumed. Corr: 0.7506
medmcqa. Corr: 0.1018


# Analysis of RAG performance per class

In [65]:
def acc(df):
    acc = np.mean(
        df.choice_answer.apply(clean_model_answer_abcde) == df.answer
    )
    return pd.Series({"accuracy": acc, "len": int(df.shape[0])})


bench = "medqa"
method = "orag-hyq-tm"


def finegrain(bench, method):
    orag = (
        dfm[method][bench]
        .groupby("class_1")
        .apply(acc)
        .sort_values("len", ascending=False)
    )

    # Now compute same for zeroshot
    zsraw = pd.read_csv(f"data/_{bench}_zeroshot.csv", sep="\t")
    zs = (
        zsraw.groupby("class_1").apply(acc).sort_values("len", ascending=False)
    )

    # Now compute same for cot
    cotraw = pd.read_csv(f"data/_{bench}_cot.csv", sep="\t")
    cot = (
        cotraw.groupby("class_1")
        .apply(acc)
        .sort_values("len", ascending=False)
    )

    # Merge all
    comp = pd.concat(
        [zs["accuracy"], cot["accuracy"], orag["accuracy"], orag["len"]],
        axis=1,
        keys=["ZeroShot", "CoT", "OntoRAG", "len"],
    ).reset_index()
    comp = comp.rename(
        columns={"class_1": "Question Class", "len": "No. Entries"}
    ).set_index(["No. Entries", "Question Class"])

    # keep only one len column
    n = 15
    display(comp.iloc[: n + 1])
    return comp.iloc[: n + 1].to_latex(float_format="%.2f")


f = finegrain("mmlumed", "orag-hyq-tm")
print(f)

/tmp/ipykernel_2570282/1867730512.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  orag = dfm[method][bench].groupby('class_1').apply(acc).sort_values('len', ascending=False)
/tmp/ipykernel_2570282/1867730512.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  zs = zsraw.groupby('class_1').apply(acc).sort_values('len', ascending=False)
/tmp/ipykernel_2570282/1867730512.py:18: DeprecationWarning: DataFra

,,ZeroShot,CoT,OntoRAG
No. Entries,Question Class,,,
405.0,Unknown,0.832099,0.777778,0.817284
311.0,Biochemistry,0.807074,0.781350,0.826367
283.0,Physiology,0.819788,0.791519,0.819788
130.0,Medicine,0.876923,0.830769,0.861538
92.0,Social & Preventive Medicine,0.750000,0.652174,0.706522
88.0,Microbiology,0.579545,0.568182,0.613636
80.0,Gynaecology & Obstetrics,0.825000,0.775000,0.825000
77.0,Anatomy,0.766234,0.766234,0.909091
72.0,Pharmacology,0.777778,0.791667,0.763889


\begin{tabular}{llrrr}
\toprule
 &  & ZeroShot & CoT & OntoRAG \\
No. Entries & Question Class &  &  &  \\
\midrule
405.000000 & Unknown & 0.83 & 0.78 & 0.82 \\
\cline{1-5}
311.000000 & Biochemistry & 0.81 & 0.78 & 0.83 \\
\cline{1-5}
283.000000 & Physiology & 0.82 & 0.79 & 0.82 \\
\cline{1-5}
130.000000 & Medicine & 0.88 & 0.83 & 0.86 \\
\cline{1-5}
92.000000 & Social & Preventive Medicine & 0.75 & 0.65 & 0.71 \\
\cline{1-5}
88.000000 & Microbiology & 0.58 & 0.57 & 0.61 \\
\cline{1-5}
80.000000 & Gynaecology & Obstetrics & 0.82 & 0.78 & 0.82 \\
\cline{1-5}
77.000000 & Anatomy & 0.77 & 0.77 & 0.91 \\
\cline{1-5}
72.000000 & Pharmacology & 0.78 & 0.79 & 0.76 \\
\cline{1-5}
68.000000 & Pediatrics & 0.85 & 0.87 & 0.85 \\
\cline{1-5}
49.000000 & Psychiatry & 0.73 & 0.76 & 0.73 \\
\cline{1-5}
33.000000 & Surgery & 0.73 & 0.67 & 0.61 \\
\cline{1-5}
23.000000 & Dental & 0.74 & 0.65 & 0.74 \\
\cline{1-5}
\multirow[t]{2}{*}{18.000000} & Genetics & 0.83 & 0.78 & 0.89 \\
 & Orthopaedics & 0.83 & 